In [1]:
import geopandas as gpd
import pandas as pd
from shapely.wkt import loads
import numpy as np
from shapely import wkt
from shapely.geometry import Point

In [2]:
csv_path = "../data/raw/ukraine_war_fires.csv"
war_fire_df = pd.read_csv(csv_path)

In [51]:
output_path = "../data/raw/output_filename.csv"
output_df = pd.read_csv(output_path)

/var/folders/6m/0w40n7p11jd2c0112ngdmyx40000gn/T/ipykernel_84519/3299804297.py:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  output_df = pd.read_csv(output_path)


In [4]:
output_df.head()

,osm_id,buildingma,addrstreet,addrcity,building,addrhousen,office,addrfull,name,buildingle,source,geometry
0,4338650.0,NaN,NaN,NaN,construction,NaN,NaN,NaN,Льодовий стадіон,5,NaN,"POLYGON ((30.4658704 50.3739959, 30.4672303 50..."
1,4528325.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4149579 45.3275258, 35.4150137 45..."
2,4528616.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4150689 45.3277749, 35.4151686 45..."
3,4528933.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4122859 45.3309186, 35.4123139 45..."
4,4529198.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((33.7198442 44.4086498, 33.720139 44...."


# Ukraine war fire dataset

In [5]:
war_fire_df.head()

,id_w_time,LATITUDE,LONGITUDE,id,x,y,year,time_of_year,ACQ_TIME,date,...,in_urban_area,pop_exact,excess_fire,predicted_fire,fire_in_window,war_fire,sustained_excess,id_big,length_of_war_fire_area,war_fire_restrictive
0,24.1-49.2-88-2022,49.210258,24.106552,24.1-49.2,24.1,49.2,2022,88,0,2022-03-29,...,True,29.594364,7,1,10,1,1,24-49,28,1
1,24.1-49.2-88-2022,49.206398,24.105635,24.1-49.2,24.1,49.2,2022,88,0,2022-03-29,...,True,14.927280,7,1,10,1,1,24-49,28,1
2,24.1-49.2-88-2022,49.210976,24.099213,24.1-49.2,24.1,49.2,2022,88,0,2022-03-29,...,True,29.594364,7,1,10,1,1,24-49,28,1
3,24.1-49.2-88-2022,49.203121,24.105589,24.1-49.2,24.1,49.2,2022,88,142,2022-03-29,...,True,14.927280,7,1,10,1,1,24-49,28,1
4,24.1-49.2-88-2022,49.211071,24.103294,24.1-49.2,24.1,49.2,2022,88,142,2022-03-29,...,True,29.594364,7,1,10,1,1,24-49,28,1


In [6]:
war_fire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42966 entries, 0 to 42965
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_w_time                42966 non-null  object 
 1   LATITUDE                 42966 non-null  float64
 2   LONGITUDE                42966 non-null  float64
 3   id                       42966 non-null  object 
 4   x                        42966 non-null  float64
 5   y                        42966 non-null  float64
 6   year                     42966 non-null  int64  
 7   time_of_year             42966 non-null  int64  
 8   ACQ_TIME                 42966 non-null  int64  
 9   date                     42966 non-null  object 
 10  fire                     42966 non-null  int64  
 11  pop_density              42966 non-null  float64
 12  city                     0 non-null      float64
 13  in_urban_area            42966 non-null  bool   
 14  pop_exact             

# Reverse locate

In [8]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="myexer")

In [9]:
war_fire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42966 entries, 0 to 42965
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_w_time                42966 non-null  object 
 1   LATITUDE                 42966 non-null  float64
 2   LONGITUDE                42966 non-null  float64
 3   id                       42966 non-null  object 
 4   x                        42966 non-null  float64
 5   y                        42966 non-null  float64
 6   year                     42966 non-null  int64  
 7   time_of_year             42966 non-null  int64  
 8   ACQ_TIME                 42966 non-null  int64  
 9   date                     42966 non-null  object 
 10  fire                     42966 non-null  int64  
 11  pop_density              42966 non-null  float64
 12  city                     0 non-null      float64
 13  in_urban_area            42966 non-null  bool   
 14  pop_exact             

In [12]:
from tqdm import tqdm

In [67]:
def find_location_buildings(row):
       
    lat, lon =row["LATITUDE"], row["LONGITUDE"]
    
    cord = f"{lat}, {lon}"
    location = geolocator.reverse(cord, exactly_one=True,language="en")
    address = location.raw['address']
    #city_keys = ['town', 'village', 'hamlet', 'municipality', 'locality']
    #city = next((address.get(key) for key in city_keys if key in address), '')
    #city = address.get('city', '')
    state = address.get('state', '')
    #country = address.get('country', '')
    
    return state
tqdm.pandas(desc="Processing")
# Assigning the state value using loc to avoid chained indexing issues
war_fire_df.loc[war_fire_df.index[5250: 10500], 'state'] = war_fire_df[5250:10500].progress_apply(find_location_buildings, axis=1)

#war_fire_df.loc[war_fire_df.index[: 10500], 'state'] = war_fire_df[:10500].apply(find_location_buildings, axis=1)

Processing: 100%|███████████████████████████| 5250/5250 [52:26<00:00,  1.67it/s]


In [68]:
war_fire_df.iloc[500: 1000]["state"].value_counts()

Odesa Oblast           177
Zhytomyr Oblast        112
Khmelnytskyi Oblast    109
Vinnytsia Oblast        63
Rivne Oblast            34
Transnistria             2
Name: state, dtype: int64

In [72]:
war_fire_df["state"].value_counts()

Kherson Oblast            3733
Mykolaiv Oblast           2548
Kyiv Oblast               1500
Odesa Oblast               941
Chernihiv Oblast           645
Ivano-Frankivsk Oblast     217
Rivne Oblast               200
Zhytomyr Oblast            181
Khmelnytskyi Oblast        119
Vinnytsia Oblast            88
Kirovohrad Oblast           86
Lviv Oblast                 82
Bryansk Oblast              51
Transnistria                47
Cherkasy Oblast             18
Chernivtsi Oblast           17
Poltava Oblast              12
Ternopil Oblast              8
Homyel Region                3
Name: state, dtype: int64

In [73]:
war_fire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42966 entries, 0 to 42965
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_w_time                42966 non-null  object 
 1   LATITUDE                 42966 non-null  float64
 2   LONGITUDE                42966 non-null  float64
 3   id                       42966 non-null  object 
 4   x                        42966 non-null  float64
 5   y                        42966 non-null  float64
 6   year                     42966 non-null  int64  
 7   time_of_year             42966 non-null  int64  
 8   ACQ_TIME                 42966 non-null  int64  
 9   date                     42966 non-null  object 
 10  fire                     42966 non-null  int64  
 11  pop_density              42966 non-null  float64
 12  city                     0 non-null      float64
 13  in_urban_area            42966 non-null  bool   
 14  pop_exact             

In [74]:
#war_fire_df.to_csv()
war_fire_df.to_csv("../data/raw/ukraine_war_fires_set_2.csv", index=False)

In [78]:
ukraine_war_fires_2_path = "../data/raw/Shikha/war_fire_df_set_all.csv"
war_fire_df_set_1 = pd.read_csv(ukraine_war_fires_2_path)

/var/folders/6m/0w40n7p11jd2c0112ngdmyx40000gn/T/ipykernel_84519/3642714246.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  war_fire_df_set_1 = pd.read_csv(ukraine_war_fires_2_path)


In [80]:
war_fire_df_set_1["state"].value_counts()

Zaporizhia Oblast                4791
Donetsk Oblast                   2671
Kharkiv Oblast                   2230
Dnipropetrovsk Oblast             636
Autonomous Republic of Crimea      88
Poltava Oblast                     68
Sumy Oblast                        11
Belgorod Oblast                     2
Name: state, dtype: int64

In [81]:
war_fire_df_set_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42966 entries, 0 to 42965
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               42966 non-null  int64  
 1   id_w_time                42966 non-null  object 
 2   LATITUDE                 42966 non-null  float64
 3   LONGITUDE                42966 non-null  float64
 4   id                       42966 non-null  object 
 5   x                        42966 non-null  float64
 6   y                        42966 non-null  float64
 7   year                     42966 non-null  int64  
 8   time_of_year             42966 non-null  int64  
 9   ACQ_TIME                 42966 non-null  int64  
 10  date                     42966 non-null  object 
 11  fire                     42966 non-null  int64  
 12  pop_density              42966 non-null  float64
 13  city                     0 non-null      float64
 14  in_urban_area         

In [82]:
war_fire_df_set_1.describe()

,Unnamed: 0,LATITUDE,LONGITUDE,x,y,year,time_of_year,ACQ_TIME,fire,pop_density,city,pop_exact,excess_fire,predicted_fire,fire_in_window,war_fire,sustained_excess,length_of_war_fire_area,war_fire_restrictive
count,42966.00000,42966.000000,42966.000000,42966.000000,42966.000000,42966.000000,42966.000000,42966.000000,42966.0,42966.000000,0.0,42966.000000,42966.000000,42966.000000,42966.000000,42966.0,42966.0,42966.000000,42966.000000
mean,21482.50000,47.885760,35.133605,35.135565,47.887357,2022.687800,191.945538,1157.957734,1.0,260.485715,NaN,486.648752,8.301936,4.974189,14.501466,1.0,1.0,519.653680,0.624238
std,12403.36017,1.253314,2.913093,2.915472,1.254211,0.463396,52.456601,652.291813,0.0,544.402538,NaN,1235.869533,12.770092,8.226906,13.920393,0.0,0.0,79.022834,0.484325
min,0.00000,45.053490,24.085392,24.100000,45.100000,2022.000000,1.000000,0.000000,1.0,1.013245,NaN,0.000000,0.000000,1.000000,1.000000,1.0,1.0,1.000000,0.000000
25%,10741.25000,46.962643,33.016245,33.000000,47.000000,2022.000000,160.000000,937.000000,1.0,12.959669,NaN,2.785603,0.000000,1.000000,6.000000,1.0,1.0,530.000000,0.000000
50%,21482.50000,47.588405,35.186955,35.200000,47.600000,2023.000000,207.000000,1034.000000,1.0,25.789587,NaN,9.763303,4.000000,1.000000,10.000000,1.0,1.0,536.000000,1.000000
75%,32223.75000,48.529930,37.834625,37.800000,48.500000,2023.000000,235.000000,1130.000000,1.0,89.725454,NaN,79.239372,10.000000,6.000000,19.000000,1.0,1.0,555.000000,1.000000
max,42965.00000,52.299830,40.022010,40.000000,52.300000,2023.000000,336.000000,2358.000000,1.0,5996.839014,NaN,9479.708008,121.000000,89.000000,124.000000,1.0,1.0,559.000000,1.000000


In [83]:
war_fire_df_state = war_fire_df_set_1[war_fire_df_set_1["state"].isin(["Odesa Oblast","Chernihiv Oblast", "Sumy Oblast", "Kharkiv Oblast","Luhansk Oblast","Donetsk Oblast","Zaporizhia Oblast","Kherson Oblast","Mykolaiv Oblast"])]

In [84]:
war_fire_df_state 

,Unnamed: 0,id_w_time,LATITUDE,LONGITUDE,id,x,y,year,time_of_year,ACQ_TIME,...,pop_exact,excess_fire,predicted_fire,fire_in_window,war_fire,sustained_excess,id_big,length_of_war_fire_area,war_fire_restrictive,state
21000,21000,35.2-47.4-234-2023,47.35305,35.24519,35.2-47.4,35.2,47.4,2023,234,917,...,3.083230,6,3,11,1,1,35-47,559,1,Zaporizhia Oblast
21001,21001,35.2-47.4-234-2023,47.35109,35.23911,35.2-47.4,35.2,47.4,2023,234,917,...,2.805319,6,3,11,1,1,35-47,559,1,Zaporizhia Oblast
21002,21002,35.2-47.4-234-2023,47.35290,35.22689,35.2-47.4,35.2,47.4,2023,234,1058,...,2.786597,6,3,11,1,1,35-47,559,1,Zaporizhia Oblast
21003,21003,35.2-47.4-234-2023,47.35352,35.23797,35.2-47.4,35.2,47.4,2023,234,917,...,2.805319,6,3,11,1,1,35-47,559,1,Zaporizhia Oblast
21004,21004,35.2-47.4-234-2023,47.35464,35.22915,35.2-47.4,35.2,47.4,2023,234,917,...,2.786597,6,3,11,1,1,35-47,559,1,Zaporizhia Oblast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31495,31495,37.8-47.2-232-2023,47.18199,37.80777,37.8-47.2,37.8,47.2,2023,232,2301,...,108.388214,31,1,34,1,1,38-47,533,1,Donetsk Oblast
31496,31496,37.8-47.2-232-2023,47.17693,37.76749,37.8-47.2,37.8,47.2,2023,232,2301,...,19.286552,31,1,34,1,1,38-47,533,1,Donetsk Oblast
31497,31497,37.8-47.2-232-2023,47.18476,37.75167,37.8-47.2,37.8,47.2,2023,232,2358,...,67.706360,31,1,34,1,1,38-47,533,1,Donetsk Oblast
31498,31498,37.8-47.2-232-2023,47.18568,37.80534,37.8-47.2,37.8,47.2,2023,232,2352,...,9.599911,31,1,34,1,1,38-47,533,1,Donetsk Oblast


In [85]:
war_fire_df_state["state"].value_counts()

Zaporizhia Oblast    4791
Donetsk Oblast       2671
Kharkiv Oblast       2230
Sumy Oblast            11
Name: state, dtype: int64

In [86]:
war_fire_df_state['geometry'] = war_fire_df_state.apply(lambda row: Point(row['LATITUDE'], row['LONGITUDE']), axis=1)

/var/folders/6m/0w40n7p11jd2c0112ngdmyx40000gn/T/ipykernel_84519/2170853753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  war_fire_df_state['geometry'] = war_fire_df_state.apply(lambda row: Point(row['LATITUDE'], row['LONGITUDE']), axis=1)


In [87]:
# ukraine_war_fires_2_path_state = "../data/raw/Shikha/war_fire_df_set_all_state_31500_42966.csv"
# war_fire_df_state = pd.read_csv(ukraine_war_fires_2_path_state)

In [88]:
war_fire_df_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9703 entries, 21000 to 31499
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               9703 non-null   int64  
 1   id_w_time                9703 non-null   object 
 2   LATITUDE                 9703 non-null   float64
 3   LONGITUDE                9703 non-null   float64
 4   id                       9703 non-null   object 
 5   x                        9703 non-null   float64
 6   y                        9703 non-null   float64
 7   year                     9703 non-null   int64  
 8   time_of_year             9703 non-null   int64  
 9   ACQ_TIME                 9703 non-null   int64  
 10  date                     9703 non-null   object 
 11  fire                     9703 non-null   int64  
 12  pop_density              9703 non-null   float64
 13  city                     0 non-null      float64
 14  in_urban_area      

In [91]:
war_fire_df_state["geometry"]

21000    POINT (47.35305 35.24519)
21001    POINT (47.35109 35.23911)
21002     POINT (47.3529 35.22689)
21003    POINT (47.35352 35.23797)
21004    POINT (47.35464 35.22915)
                   ...            
31495    POINT (47.18199 37.80777)
31496    POINT (47.17693 37.76749)
31497    POINT (47.18476 37.75167)
31498    POINT (47.18568 37.80534)
31499    POINT (47.18278 37.76458)
Name: geometry, Length: 9703, dtype: object

# calculate UTM

In [92]:
import geopandas as gpd
# change to GeoDataFrame
# geo_build_df = gpd.GeoDataFrame(buildings_df)
# geo_build_df = geo_build_df.set_geometry("geometry_wkt_cent")

In [93]:
# from pyproj import Proj
# pp = Proj(proj='utm',zone=10,ellps='WGS84', preserve_units=False)

# geo_build_df["xx"], geo_build_df["yy"]= pp(geo_build_df["geometry_wkt_cent"].x.values,geo_build_df["geometry_wkt_cent"].y.values)
# # My_data["X"] = xx
# # My_data["Y"] = yy 

In [94]:
# First, convert WKT column to geometries
#buildings_df_state['geometry_wkt_cent2'] = gpd.GeoSeries.from_wkt(buildings_df_state['geometry_wkt_cent'])

# Then create a GeoDataFrame, setting the geometry to the converted geometries
# geo_build_df = gpd.GeoDataFrame(buildings_df_state, geometry='geometry_wkt_cent')

In [95]:
# geo_build_df.crs = 'epsg:32635'
# # change the projection of geodf
# geo_build_df_2 = geo_build_df.to_crs(epsg=32635)
# print(geo_build_df_2)

In [96]:
#geo_build_df_2["geometry_wkt_cent"]

# Buffer 

In [97]:
# import geopandas as gpd
# from shapely.geometry import Point
# from pprint import pprint

# buffer_radius_m = 500
# buffered_gdf = geo_build_df_2.copy()

# buffered_gdf['geometry_wkt_cent_buff'] = buffered_gdf.buffer(buffer_radius_m)  # 1 degree of latitude is approximately 111.32 km
# # Print the buffered GeoDataFrame

# pprint(buffered_gdf)

In [98]:
#pip install pyproj --upgrade


In [99]:
import pyproj
print(pyproj.__version__)


3.6.1


### with utm conversion


In [100]:
# First, find the UTM zone for the geometries and project to that CRS
# .estimate_utm_crs() is available in GeoPandas 0.9.0 and later
geo_warfires_df = gpd.GeoDataFrame(war_fire_df_state, geometry='geometry')
geo_warfires_df.set_crs(epsg=4326, inplace=True)
utm_crs = geo_warfires_df.estimate_utm_crs()
geo_warfires_df_utm = geo_warfires_df.to_crs("EPSG:32635")

# Then, apply the buffer in meters
buffer_radius_m = 500  # Buffer radius in meters
geo_warfires_df_utm['geometry_buff'] = geo_warfires_df_utm.buffer(buffer_radius_m)

In [101]:
geo_warfires_df_utm['geometry_buff']

21000    POLYGON ((2364867.654 4096193.819, 2364865.247...
21001    POLYGON ((2364833.034 4095467.092, 2364830.626...
21002    POLYGON ((2365296.972 4094120.691, 2365294.564...
21003    POLYGON ((2365086.171 4095386.422, 2365083.764...
21004    POLYGON ((2365403.753 4094410.929, 2365401.345...
                               ...                        
31495    POLYGON ((2285741.100 4381865.942, 2285738.692...
31496    POLYGON ((2286310.218 4377232.094, 2286307.810...
31497    POLYGON ((2287410.463 4375609.500, 2287408.055...
31498    POLYGON ((2286132.124 4381666.793, 2286129.716...
31499    POLYGON ((2286906.483 4377022.321, 2286904.075...
Name: geometry_buff, Length: 9703, dtype: geometry

In [102]:
geo_warfires_df_utm = geo_warfires_df_utm.set_geometry("geometry_buff")

In [103]:
geo_warfires_df_utm.head()

,Unnamed: 0,id_w_time,LATITUDE,LONGITUDE,id,x,y,year,time_of_year,ACQ_TIME,...,predicted_fire,fire_in_window,war_fire,sustained_excess,id_big,length_of_war_fire_area,war_fire_restrictive,state,geometry,geometry_buff
21000,21000,35.2-47.4-234-2023,47.35305,35.24519,35.2-47.4,35.2,47.4,2023,234,917,...,3,11,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2364367.654 4096193.819),"POLYGON ((2364867.654 4096193.819, 2364865.247..."
21001,21001,35.2-47.4-234-2023,47.35109,35.23911,35.2-47.4,35.2,47.4,2023,234,917,...,3,11,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2364333.034 4095467.092),"POLYGON ((2364833.034 4095467.092, 2364830.626..."
21002,21002,35.2-47.4-234-2023,47.35290,35.22689,35.2-47.4,35.2,47.4,2023,234,1058,...,3,11,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2364796.972 4094120.691),"POLYGON ((2365296.972 4094120.691, 2365294.564..."
21003,21003,35.2-47.4-234-2023,47.35352,35.23797,35.2-47.4,35.2,47.4,2023,234,917,...,3,11,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2364586.171 4095386.422),"POLYGON ((2365086.171 4095386.422, 2365083.764..."
21004,21004,35.2-47.4-234-2023,47.35464,35.22915,35.2-47.4,35.2,47.4,2023,234,917,...,3,11,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2364903.753 4094410.929),"POLYGON ((2365403.753 4094410.929, 2365401.345..."


In [104]:
type(geo_warfires_df.iloc[0]["LATITUDE"])

numpy.float64

### without utm conversion
#### Don't execute this if you already did the utm

In [114]:
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon
geo_warfires_df = gpd.GeoDataFrame(war_fire_df_state, geometry='geometry')
# Function to create a buffer polygon around a point
def geodesic_point_buffer(lat, lon, radius):
    center = Point(lon, lat)
    # Generate the points for the buffer polygon
    points = [
        geodesic(kilometers=radius/1000).destination((lat, lon), bearing).format_decimal()
        for bearing in range(0, 360)
    ]
    # Convert the string coordinates to tuples of floats (lon, lat)
    points = [tuple(map(float, p.split(','))) for p in points]
    # Create a polygon with the points
    return Polygon(points)

buffer_radius_m = 500
# Apply the function to each point in our GeoDataFrame
geo_warfires_df['geometry_buff'] = geo_warfires_df.apply(
    lambda x: geodesic_point_buffer(x['LATITUDE'], x['LONGITUDE'], buffer_radius_m), axis=1
)

In [115]:
geo_warfires_df['geometry_buff']

647    POLYGON ((45.55535 28.69054, 45.55535 28.69065...
648    POLYGON ((45.55909 28.69310, 45.55909 28.69321...
649    POLYGON ((45.55587 28.69108, 45.55587 28.69119...
650    POLYGON ((45.55942 28.69486, 45.55942 28.69497...
651    POLYGON ((45.56280 28.69397, 45.56280 28.69408...
                             ...                        
982    POLYGON ((45.98861 29.32994, 45.98861 29.33006...
983    POLYGON ((45.95480 29.29104, 45.95480 29.29115...
984    POLYGON ((47.62736 29.27261, 47.62736 29.27273...
985    POLYGON ((47.69050 29.26837, 47.69050 29.26849...
986    POLYGON ((47.68073 29.27690, 47.68073 29.27702...
Name: geometry_buff, Length: 177, dtype: geometry

In [113]:

# # Function to create a buffer polygon around a point
# def geodesic_point_buffer(lat, lon, radius):
#     point = geodesic(kilometers=radius/1000.0).destination(Point(lon, lat), bearing=0)
#     return Polygon([(geodesic(kilometers=radius/1000.0).destination(Point(lon, lat), bearing=bearing).longitude, geodesic(kilometers=radius/1000.0).destination(Point(lon, lat), bearing=bearing).latitude) for bearing in range(0, 360)])
# buffer_radius_m = 500
# # Apply the function to each point in your GeoDataFrame
# geo_warfires_df['geometry_buff'] = geo_warfires_df.apply(lambda x: geodesic_point_buffer(x['LATITUDE'], x['LONGITUDE'], buffer_radius_m), axis=1)


TypeError: Failed to create Point instance from <POINT (28.7 45.6)>.

# Buildings dataset

## Convert string geometry to integer

In [105]:
output_df['geometry_wkt'] = output_df['geometry'].apply(lambda x: loads(x) if x is not None else None)

In [106]:
#output_df.to_csv("output_filename_wkt.csv", index=False)

In [107]:
# output_path = "../data/raw/output_filename_wkt.csv"
# output_df = pd.read_csv(output_path)

In [108]:
buildings_df = output_df.copy()
buildings_df.head()

,osm_id,buildingma,addrstreet,addrcity,building,addrhousen,office,addrfull,name,buildingle,source,geometry,geometry_wkt
0,4338650.0,NaN,NaN,NaN,construction,NaN,NaN,NaN,Льодовий стадіон,5,NaN,"POLYGON ((30.4658704 50.3739959, 30.4672303 50...","POLYGON ((30.4658704 50.3739959, 30.4672303 50..."
1,4528325.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4149579 45.3275258, 35.4150137 45...","POLYGON ((35.4149579 45.3275258, 35.4150137 45..."
2,4528616.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4150689 45.3277749, 35.4151686 45...","POLYGON ((35.4150689 45.3277749, 35.4151686 45..."
3,4528933.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4122859 45.3309186, 35.4123139 45...","POLYGON ((35.4122859 45.3309186, 35.4123139 45..."
4,4529198.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((33.7198442 44.4086498, 33.720139 44....","POLYGON ((33.7198442 44.4086498, 33.720139 44...."


In [109]:
print(buildings_df["buildingle"].isna().sum()/len(buildings_df))

0.845716634706276


In [110]:
buildings_df["geometry_wkt_cent"] = np.nan

In [111]:
# buildings_df["geometry_wkt_cent"] = buildings_df["geometry_wkt"].apply(lambda x: x.centroid )
buildings_df['geometry_wkt_cent'] = buildings_df['geometry_wkt'].apply(lambda geom: Point(geom.centroid.y, geom.centroid.x))

# This will result in a column 'centroid' with tuples where the first element is latitude and the second is longitude.

In [113]:
buildings_df["geometry_wkt_cent"][21000:21100]

21000    POINT (48.50416405264397 35.07598153263674)
21001    POINT (48.49566274515011 35.06945856951236)
21002    POINT (48.49222352135427 35.07691937071695)
21003                POINT (48.51255865 35.06411975)
21004    POINT (48.50155048293292 35.08471032890138)
                            ...                     
21095                  POINT (48.4609056 34.9361139)
21096           POINT (48.44330010000001 34.9208926)
21097    POINT (48.45685862362831 34.93766187668012)
21098                 POINT (48.45692125 34.9370002)
21099    POINT (48.45740824340668 34.93552876278028)
Name: geometry_wkt_cent, Length: 100, dtype: object

## Find the intersection of war_fires and buildings


### Buildings dataset with utm conversion


In [114]:
# First, find the UTM zone for the geometries and project to that CRS
# .estimate_utm_crs() is available in GeoPandas 0.9.0 and later
geo_buildings_df = gpd.GeoDataFrame(buildings_df, geometry='geometry_wkt_cent')
geo_buildings_df.set_crs(epsg=4326, inplace=True)
#utm_crs = geo_warfires_df.estimate_utm_crs()
geo_buildings_df_utm = geo_buildings_df.to_crs("EPSG:32635")

# Then, apply the buffer in meters
#buffer_radius_m = 500  # Buffer radius in meters
#geo_warfires_df_utm['geometry_buff'] = geo_warfires_df_utm.buffer(buffer_radius_m)

In [115]:
#geo_warfires_df_utm['geometry_buff']

### How many buildings are within 500 meters of warfires

In [116]:
#geo_buildings_df = gpd.GeoDataFrame(buildings_df_utm, geometry='geometry_wkt_cent')

In [117]:
# Function to count buildings within a given buffer polygon
def count_buildings_in_buffer(buffer, buildings_gdf):
    # Use the within method to check if buildings are within the buffer zone
    return buildings_gdf.within(buffer).sum()

# Apply the function to each buffer zone and add the result as a new column
geo_warfires_df_utm['building_count'] = geo_warfires_df_utm['geometry_buff'].apply(
    lambda buffer: count_buildings_in_buffer(buffer, geo_buildings_df_utm)
)

In [118]:
geo_warfires_df_utm['geometry_buff']

21000    POLYGON ((2364867.654 4096193.819, 2364865.247...
21001    POLYGON ((2364833.034 4095467.092, 2364830.626...
21002    POLYGON ((2365296.972 4094120.691, 2365294.564...
21003    POLYGON ((2365086.171 4095386.422, 2365083.764...
21004    POLYGON ((2365403.753 4094410.929, 2365401.345...
                               ...                        
31495    POLYGON ((2285741.100 4381865.942, 2285738.692...
31496    POLYGON ((2286310.218 4377232.094, 2286307.810...
31497    POLYGON ((2287410.463 4375609.500, 2287408.055...
31498    POLYGON ((2286132.124 4381666.793, 2286129.716...
31499    POLYGON ((2286906.483 4377022.321, 2286904.075...
Name: geometry_buff, Length: 9703, dtype: geometry

In [119]:
geo_warfires_df_utm.head()

,Unnamed: 0,id_w_time,LATITUDE,LONGITUDE,id,x,y,year,time_of_year,ACQ_TIME,...,fire_in_window,war_fire,sustained_excess,id_big,length_of_war_fire_area,war_fire_restrictive,state,geometry,geometry_buff,building_count
21000,21000,35.2-47.4-234-2023,47.35305,35.24519,35.2-47.4,35.2,47.4,2023,234,917,...,11,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2364367.654 4096193.819),"POLYGON ((2364867.654 4096193.819, 2364865.247...",0
21001,21001,35.2-47.4-234-2023,47.35109,35.23911,35.2-47.4,35.2,47.4,2023,234,917,...,11,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2364333.034 4095467.092),"POLYGON ((2364833.034 4095467.092, 2364830.626...",0
21002,21002,35.2-47.4-234-2023,47.35290,35.22689,35.2-47.4,35.2,47.4,2023,234,1058,...,11,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2364796.972 4094120.691),"POLYGON ((2365296.972 4094120.691, 2365294.564...",0
21003,21003,35.2-47.4-234-2023,47.35352,35.23797,35.2-47.4,35.2,47.4,2023,234,917,...,11,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2364586.171 4095386.422),"POLYGON ((2365086.171 4095386.422, 2365083.764...",0
21004,21004,35.2-47.4-234-2023,47.35464,35.22915,35.2-47.4,35.2,47.4,2023,234,917,...,11,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2364903.753 4094410.929),"POLYGON ((2365403.753 4094410.929, 2365401.345...",0


In [120]:
geo_warfires_df_utm[geo_warfires_df_utm['building_count']  > 0]

,Unnamed: 0,id_w_time,LATITUDE,LONGITUDE,id,x,y,year,time_of_year,ACQ_TIME,...,fire_in_window,war_fire,sustained_excess,id_big,length_of_war_fire_area,war_fire_restrictive,state,geometry,geometry_buff,building_count
21008,21008,35.2-47.9-106-2023,47.868890,35.163540,35.2-47.9,35.2,47.9,2023,106,2314,...,8,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2414292.740 4097346.679),"POLYGON ((2414792.740 4097346.679, 2414790.332...",10
21009,21009,35.2-47.9-106-2023,47.878710,35.192700,35.2-47.9,35.2,47.9,2023,106,2314,...,8,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2414480.001 4100849.540),"POLYGON ((2414980.001 4100849.540, 2414977.594...",21
21010,21010,35.2-47.9-106-2023,47.880850,35.193360,35.2-47.9,35.2,47.9,2023,106,24,...,8,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2414662.495 4100968.033),"POLYGON ((2415162.495 4100968.033, 2415160.088...",24
21011,21011,35.2-47.9-106-2023,47.869360,35.158180,35.2-47.9,35.2,47.9,2023,106,2314,...,8,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2414469.773 4096749.277),"POLYGON ((2414969.773 4096749.277, 2414967.365...",12
21012,21012,35.2-47.9-106-2023,47.879180,35.187380,35.2-47.9,35.2,47.9,2023,106,2314,...,8,1,1,35-47,559,1,Zaporizhia Oblast,POINT (2414656.180 4100256.702),"POLYGON ((2415156.180 4100256.702, 2415153.772...",23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31433,31433,37.8-47.1-241-2023,47.120490,37.753910,37.8-47.1,37.8,47.1,2023,241,1028,...,10,1,1,38-47,533,0,Donetsk Oblast,POINT (2281112.029 4374570.973),"POLYGON ((2281612.029 4374570.973, 2281609.622...",3
31448,31448,37.8-47.1-82-2022,47.092159,37.778599,37.8-47.1,37.8,47.1,2022,82,2354,...,22,1,1,38-47,533,0,Donetsk Oblast,POINT (2277958.685 4376781.207),"POLYGON ((2278458.685 4376781.207, 2278456.278...",51
31455,31455,37.8-47.1-82-2022,47.139700,37.847400,37.8-47.1,37.8,47.1,2022,82,904,...,22,1,1,38-47,533,0,Donetsk Oblast,POINT (2280463.106 4385474.395),"POLYGON ((2280963.106 4385474.395, 2280960.698...",2
31456,31456,37.8-47.1-82-2022,47.100900,37.793600,37.8-47.1,37.8,47.1,2022,82,1055,...,22,1,1,38-47,533,0,Donetsk Oblast,POINT (2278360.666 4378643.884),"POLYGON ((2278860.666 4378643.884, 2278858.259...",2


In [121]:
geo_warfires_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 9703 entries, 21000 to 31499
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Unnamed: 0               9703 non-null   int64   
 1   id_w_time                9703 non-null   object  
 2   LATITUDE                 9703 non-null   float64 
 3   LONGITUDE                9703 non-null   float64 
 4   id                       9703 non-null   object  
 5   x                        9703 non-null   float64 
 6   y                        9703 non-null   float64 
 7   year                     9703 non-null   int64   
 8   time_of_year             9703 non-null   int64   
 9   ACQ_TIME                 9703 non-null   int64   
 10  date                     9703 non-null   object  
 11  fire                     9703 non-null   int64   
 12  pop_density              9703 non-null   float64 
 13  city                     0 non-null      float64 


In [122]:
geo_warfires_df_utm.to_csv("../data/raw/shikha/war_fires_build_count_set_shikha.csv", index=False)

# Combine the build count dataframes
#### created by Me, Shika, Vivian and Anjali

In [ ]:
# Load Chiff

In [123]:
csv_path = "../data/raw/war_fires_build_count_set_1.csv"
geo_warfires_df_utm_1 = pd.read_csv(csv_path)

In [124]:
csv_path = "../data/raw/war_fires_build_count_set_2.csv"
geo_warfires_df_utm_2 = pd.read_csv(csv_path)

In [125]:
# Load Vivian dataset

In [126]:
csv_path = "../data/raw/viv/war_fire_df_set_all_state.csv"
geo_warfires_df_utm_viv = pd.read_csv(csv_path)

In [ ]:
# Load Shikha dataset

In [127]:
csv_path = "../data/raw/shikha/war_fires_build_count_set_shikha.csv"
geo_warfires_df_utm_shikha = pd.read_csv(csv_path)

In [ ]:
# Load Anjali dataset

In [128]:
csv_path = "../data/raw/anjali/war_fires_build_count_set_anjali.csv"
geo_warfires_df_utm_anjali = pd.read_csv(csv_path)

In [129]:
geo_warfires_df_utm_anjali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11456 entries, 0 to 11455
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               11456 non-null  int64  
 1   id_w_time                11456 non-null  object 
 2   LATITUDE                 11456 non-null  float64
 3   LONGITUDE                11456 non-null  float64
 4   id                       11456 non-null  object 
 5   x                        11456 non-null  float64
 6   y                        11456 non-null  float64
 7   year                     11456 non-null  int64  
 8   time_of_year             11456 non-null  int64  
 9   ACQ_TIME                 11456 non-null  int64  
 10  date                     11456 non-null  object 
 11  fire                     11456 non-null  int64  
 12  pop_density              11456 non-null  float64
 13  city                     0 non-null      float64
 14  in_urban_area         

In [130]:
result_df = pd.concat([geo_warfires_df_utm_1, geo_warfires_df_utm_2,geo_warfires_df_utm_viv, geo_warfires_df_utm_shikha, geo_warfires_df_utm_anjali], ignore_index=True)


In [131]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39810 entries, 0 to 39809
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_w_time                39810 non-null  object 
 1   LATITUDE                 39810 non-null  float64
 2   LONGITUDE                39810 non-null  float64
 3   id                       39810 non-null  object 
 4   x                        39810 non-null  float64
 5   y                        39810 non-null  float64
 6   year                     39810 non-null  int64  
 7   time_of_year             39810 non-null  int64  
 8   ACQ_TIME                 39810 non-null  int64  
 9   date                     39810 non-null  object 
 10  fire                     39810 non-null  int64  
 11  pop_density              39810 non-null  float64
 12  city                     0 non-null      float64
 13  in_urban_area            39810 non-null  bool   
 14  pop_exact             

In [134]:
result_df[result_df['building_count']  > 0]

,id_w_time,LATITUDE,LONGITUDE,id,x,y,year,time_of_year,ACQ_TIME,date,...,war_fire,sustained_excess,id_big,length_of_war_fire_area,war_fire_restrictive,state,geometry,geometry_buff,building_count,Unnamed: 0
8,28.9-45.3-214-2023,45.302940,28.894770,28.9-45.3,28.9,45.3,2023,214,2339,2023-08-02,...,1,1,29-45,2,1,Odesa Oblast,POINT (2300755.723462406 3338415.494418401),"POLYGON ((2301255.723462406 3338415.494418401,...",21,NaN
9,28.9-45.3-214-2023,45.306980,28.893870,28.9-45.3,28.9,45.3,2023,214,48,2023-08-02,...,1,1,29-45,2,1,Odesa Oblast,POINT (2301176.732781155 3338377.773124649),"POLYGON ((2301676.732781155 3338377.773124649,...",47,NaN
10,28.9-45.3-214-2023,45.304500,28.895480,28.9-45.3,28.9,45.3,2023,214,1123,2023-08-02,...,1,1,29-45,2,1,Odesa Oblast,POINT (2300899.043343316 3338521.282572986),"POLYGON ((2301399.043343316 3338521.282572986,...",40,NaN
11,28.9-45.3-214-2023,45.300620,28.898450,28.9-45.3,28.9,45.3,2023,214,48,2023-08-02,...,1,1,29-45,2,1,Odesa Oblast,POINT (2300456.403102858 3338797.153019492),"POLYGON ((2300956.403102858 3338797.153019492,...",9,NaN
12,28.9-45.3-214-2023,45.305150,28.900790,28.9-45.3,28.9,45.3,2023,214,48,2023-08-02,...,1,1,29-45,2,1,Odesa Oblast,POINT (2300867.490339371 3339136.015479554),"POLYGON ((2301367.490339371 3339136.015479554,...",17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39547,39.7-47.9-227-2022,47.854088,39.731686,39.7-47.9,39.7,47.9,2022,227,942,2022-08-15,...,1,1,40-48,527,0,Luhansk Oblast,POINT (2293676.074004888 4611825.875171651),"POLYGON ((2294176.074004888 4611825.875171651,...",1,42703.0
39558,39.7-48-243-2023,48.007210,39.657650,39.7-48,39.7,48.0,2023,243,900,2023-08-31,...,1,1,40-48,527,0,Luhansk Oblast,POINT (2308971.131077449 4606764.685206224),"POLYGON ((2309471.131077449 4606764.685206224,...",1,42714.0
39604,39.7-48.5-246-2023,48.541620,39.725560,39.7-48.5,39.7,48.5,2023,246,722,2023-09-03,...,1,1,40-48,527,1,Luhansk Oblast,POINT (2353390.922560195 4625904.011682282),"POLYGON ((2353890.922560195 4625904.011682282,...",2,42760.0
39648,39.8-48.1-85-2022,48.065400,39.811800,39.8-48.1,39.8,48.1,2022,85,756,2022-03-26,...,1,1,40-48,527,1,Luhansk Oblast,POINT (2309758.161004319 4625291.237293408),"POLYGON ((2310258.161004319 4625291.237293408,...",11,42804.0


In [135]:
result_df.to_csv("../data/raw/war_fires_build_count_set_full.csv", index=False)

# Count the buildings for each state

In [2]:
result_df_path= "../data/raw/war_fires_build_count_set_full.csv"
result_df = pd.read_csv(result_df_path)

In [3]:
result_df.head()

,id_w_time,LATITUDE,LONGITUDE,id,x,y,year,time_of_year,ACQ_TIME,date,...,war_fire,sustained_excess,id_big,length_of_war_fire_area,war_fire_restrictive,state,geometry,geometry_buff,building_count,Unnamed: 0
0,28.7-45.6-239-2023,45.55085,28.69054,28.7-45.6,28.7,45.6,2023,239,1104,2023-08-27,...,1,1,29-46,531,1,Odesa Oblast,POINT (2329355.658988406 3319152.759123723),"POLYGON ((2329855.658988406 3319152.759123723,...",0,NaN
1,28.7-45.6-239-2023,45.55459,28.69310,28.7-45.6,28.7,45.6,2023,239,1015,2023-08-27,...,1,1,29-46,531,1,Odesa Oblast,POINT (2329684.345463021 3319504.96926707),"POLYGON ((2330184.345463021 3319504.96926707, ...",0,NaN
2,28.7-45.6-239-2023,45.55137,28.69108,28.7-45.6,28.7,45.6,2023,239,1155,2023-08-27,...,1,1,29-46,531,1,Odesa Oblast,POINT (2329397.980036283 3319222.69775088),"POLYGON ((2329897.980036283 3319222.69775088, ...",0,NaN
3,28.7-45.6-239-2023,45.55492,28.69486,28.7-45.6,28.7,45.6,2023,239,1104,2023-08-27,...,1,1,29-46,531,1,Odesa Oblast,POINT (2329685.16681294 3319710.821818586),"POLYGON ((2330185.16681294 3319710.821818586, ...",0,NaN
4,28.7-45.6-239-2023,45.55830,28.69397,28.7-45.6,28.7,45.6,2023,239,1104,2023-08-27,...,1,1,29-46,531,1,Odesa Oblast,POINT (2330041.047240355 3319664.177652),"POLYGON ((2330541.047240355 3319664.177652, 23...",0,NaN


In [6]:
df_state_build_count = result_df.groupby("state").sum("building_count")

In [9]:
df_state_build_count

,LATITUDE,LONGITUDE,x,y,year,time_of_year,ACQ_TIME,fire,pop_density,city,...,pop_exact,excess_fire,predicted_fire,fire_in_window,war_fire,sustained_excess,length_of_war_fire_area,war_fire_restrictive,building_count,Unnamed: 0
state,,,,,,,,,,,,,,,,,,,,,
Chernihiv Oblast,50383.764982,30893.721888,30895.1,50381.3,1971921,106901,976556,975,5.996761e+05,0.0,...,6.227482e+05,5517,5733,12310,975,975,377270,484,18755,0.0
Donetsk Oblast,392381.407514,310283.677961,310298.5,392377.4,16572710,1608647,9694823,8194,1.638508e+06,0.0,...,2.458813e+06,77341,48386,137147,8194,8194,4406043,4887,120503,270513749.0
Kharkiv Oblast,115332.498970,86396.757948,86402.9,115340.2,4716473,344184,2706307,2332,8.688918e+04,0.0,...,9.623282e+04,13891,11354,28957,2332,2332,1218923,1495,4486,65682794.0
Kherson Oblast,495750.382515,353346.659555,353358.9,495738.3,21478336,2206982,11894858,10618,5.164498e+05,0.0,...,4.573362e+05,136274,22527,177390,10618,10618,5671388,7903,15900,93324296.0
Luhansk Oblast,284547.706904,226484.123279,226556.9,284618.8,11830244,1052097,7764682,5849,3.637039e+06,0.0,...,9.102067e+06,39144,47217,91404,5849,5849,3048063,3364,136646,232306241.0
Mykolaiv Oblast,173155.500015,118142.450310,118138.3,173151.4,7449351,737801,3993077,3683,1.210911e+05,0.0,...,1.677696e+05,33727,14643,54306,3683,3683,1969634,2468,3966,1509997.0
Odesa Oblast,87787.461252,56150.911456,56152.0,87791.4,3807026,392520,1955364,1882,6.644771e+04,0.0,...,4.492936e+04,9958,2882,16084,1882,1882,927380,1436,1218,0.0
Sumy Oblast,7960.546831,5305.923973,5305.8,7961.7,315587,19323,168873,156,3.441083e+03,0.0,...,1.011892e+03,1282,163,1745,156,156,21792,138,0,2469085.0
Zaporizhia Oblast,289709.140493,218414.097744,218425.1,289725.8,12381451,1355296,6904650,6121,1.425496e+06,0.0,...,4.429304e+06,35799,25822,69023,6121,6121,3236024,3671,13335,140741435.0


In [7]:
df_state_build_count.to_csv("../data/raw/df_state_build_count.csv", index=False)

#                     STOP HERE
### THE build count dataset creation ends here #######

# UTM conversion

In [64]:
# import geopandas as gpd
# from shapely.geometry import Point
# from pprint import pprint

# buffer_radius_m = 500
# buffered_gdf = geo_build_df_2.copy()

# buffered_gdf['geometry_wkt_cent_buff'] = buffered_gdf.buffer(buffer_radius_m)  # 1 degree of latitude is approximately 111.32 km
# # Print the buffered GeoDataFrame

# pprint(buffered_gdf)

In [65]:
# pip install pyproj --upgrade


In [66]:
# import pyproj
# print(pyproj.__version__)


In [89]:
# First, find the UTM zone for the geometries and project to that CRS
# .estimate_utm_crs() is available in GeoPandas 0.9.0 and later
geo_warfires_df = gpd.GeoDataFrame(buildings_df, geometry='geometry_wkt_cent')
geo_build_df.set_crs(epsg=4326, inplace=True)
utm_crs = geo_build_df.estimate_utm_crs()
geo_build_df_utm = geo_build_df.to_crs(utm_crs)

# Then, apply the buffer in meters
# buffer_radius_m = 500  # Buffer radius in meters
# geo_build_df_utm['geometry_wkt_cent_buff'] = geo_build_df_utm.buffer(buffer_radius_m)

In [90]:
geo_build_df_utm['geometry_wkt_cent']

990    POINT (421919.847 5346779.359)
991    POINT (421753.849 5346692.502)
992    POINT (421824.872 5346723.293)
993    POINT (421823.681 5346682.403)
994    POINT (421781.523 5346663.285)
995    POINT (421689.345 5346680.499)
996    POINT (421646.813 5346660.227)
997    POINT (421652.919 5346630.706)
998    POINT (421562.532 5346570.624)
999    POINT (421571.570 5346631.445)
Name: geometry_wkt_cent, dtype: geometry

In [95]:
geo_build_df_utm = geo_build_df_utm.to_crs(geo_warfires_df_utm.crs)

In [96]:
print(geo_build_df_utm.crs)
print(geo_warfires_df_utm.crs)

EPSG:32635
EPSG:32635


In [97]:
# Assuming your buildings GeoDataFrame is called 'geo_build_df_utm' and it's already in the same CRS as 'geo_warfires_df_utm'
# Perform the spatial join
joined_gdf = gpd.sjoin(geo_build_df_utm, geo_warfires_df_utm, op='within')

print(joined_gdf.head())
# Count the number of buildings in each polygon
building_counts = joined_gdf.groupby('index_right').size()
print(building_counts)

# # Now, let's add the building counts to our 'geo_warfires_df_utm' GeoDataFrame
# # Assuming 'polygon_id' is our unique identifier for the polygons
# geo_warfires_df_utm['building_count'] = geo_warfires_df_utm['polygon_id'].map(building_counts)

# # Fill NaN values with 0 where there are no buildings in the polygon
# geo_warfires_df_utm['building_count'].fillna(0, inplace=True)

# # If you don't have a unique identifier, use the index
# if 'polygon_id' not in geo_warfires_df_utm:
#     geo_warfires_df_utm['building_count'] = building_counts.reindex(geo_warfires_df_utm.index, fill_value=0)

Empty GeoDataFrame
Columns: [osm_id, buildingma, addrstreet, addrcity, building, addrhousen, office, addrfull, name, buildingle, source, geometry_left, geometry_wkt, geometry_wkt_cent, state_left, index_right, id_w_time, LATITUDE, LONGITUDE, id, x, y, year, time_of_year, ACQ_TIME, date, fire, pop_density, city, in_urban_area, pop_exact, excess_fire, predicted_fire, fire_in_window, war_fire, sustained_excess, id_big, length_of_war_fire_area, war_fire_restrictive, state_right, geometry_right]
Index: []

[0 rows x 41 columns]
Series([], dtype: int64)


/Users/Chiffon/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [93]:
print(geo_build_df_utm.is_valid)
print(geo_warfires_df_utm.is_valid)

990    True
991    True
992    True
993    True
994    True
995    True
996    True
997    True
998    True
999    True
dtype: bool
647    True
648    True
649    True
650    True
651    True
       ... 
982    True
983    True
984    True
985    True
986    True
Length: 177, dtype: bool


In [94]:
print(geo_build_df_utm.total_bounds)
print(geo_warfires_df_utm.total_bounds)

[ 421562.53249575 5346570.62385328  421919.84713465 5346779.35884353]
[2299956.40310286 3318606.09706398 2545458.31056401 3423527.70621669]


# Reverse locate

In [20]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="myexer")

In [21]:
def find_location_buildings(row):
    point = row["geometry_wkt_cent"]
    print(point)
    lat, lon = point.y, point.x
    
    cord = f"{lat}, {lon}"
    location = geolocator.reverse(cord, exactly_one=True,language="en")
    address = location.raw['address']
    #city_keys = ['town', 'village', 'hamlet', 'municipality', 'locality']
    #city = next((address.get(key) for key in city_keys if key in address), '')
    #city = address.get('city', '')
    state = address.get('state', '')
    country = address.get('country', '')
    
    return state

# Assigning the state value using loc to avoid chained indexing issues
buildings_df.loc[buildings_df.index[990: 1000], 'state'] = buildings_df.iloc[990:1000].apply(find_location_buildings, axis=1)

POINT (25.94778949260989 48.26940593569441)
POINT (25.94556896352371 48.26860415107055)
POINT (25.94652019729562 48.26888988885725)
POINT (25.94651171412588 48.26852193096251)
POINT (25.94594723050538 48.26834475095112)
POINT (25.94470209284609 48.26848820481244)
POINT (25.94413280550882 48.26830059335505)
POINT (25.94422053876138 48.26803580531003)
POINT (25.94301388236103 48.26748416014559)
POINT (25.94312437031487 48.2680323818675)


In [22]:
buildings_df.iloc[989:1001]

,osm_id,buildingma,addrstreet,addrcity,building,addrhousen,office,addrfull,name,buildingle,source,geometry,geometry_wkt,geometry_wkt_cent,state
989,26735539.0,NaN,NaN,NaN,apartments,16,NaN,NaN,NaN,10,NaN,"POLYGON ((35.0603633 48.4132264, 35.0605043 48...","POLYGON ((35.0603633 48.4132264, 35.0605043 48...",NaN,NaN
990,26742800.0,NaN,NaN,NaN,yes,91а,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9476411 48.2694433, 25.9477351 48...","POLYGON ((25.9476411 48.2694433, 25.9477351 48...",POINT (25.94778949260989 48.26940593569441),Chernivtsi Oblast
991,26742866.0,NaN,NaN,NaN,yes,103,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9453659 48.2687344, 25.9455505 48...","POLYGON ((25.9453659 48.2687344, 25.9455505 48...",POINT (25.94556896352371 48.26860415107055),Chernivtsi Oblast
992,26742872.0,NaN,NaN,NaN,yes,95,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9463234 48.2690174, 25.9465101 48...","POLYGON ((25.9463234 48.2690174, 25.9465101 48...",POINT (25.94652019729562 48.26888988885725),Chernivtsi Oblast
993,26742930.0,NaN,NaN,NaN,yes,97,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9462377 48.2685023, 25.9463827 48...","POLYGON ((25.9462377 48.2685023, 25.9463827 48...",POINT (25.94651171412588 48.26852193096251),Chernivtsi Oblast
994,26742953.0,NaN,NaN,NaN,yes,101,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9456757 48.2683202, 25.9458197 48...","POLYGON ((25.9456757 48.2683202, 25.9458197 48...",POINT (25.94594723050538 48.26834475095112),Chernivtsi Oblast
995,26742991.0,NaN,NaN,NaN,yes,107,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9442038 48.2683966, 25.9451136 48...","POLYGON ((25.9442038 48.2683966, 25.9451136 48...",POINT (25.94470209284609 48.26848820481244),Chernivtsi Oblast
996,26742992.0,NaN,Івана Миколайчука вулиця,NaN,yes,1,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9439751 48.2683212, 25.9442038 48...","POLYGON ((25.9439751 48.2683212, 25.9442038 48...",POINT (25.94413280550882 48.26830059335505),Chernivtsi Oblast
997,26742997.0,NaN,Івана Миколайчука вулиця,NaN,yes,3,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9440047 48.2681858, 25.9440618 48...","POLYGON ((25.9440047 48.2681858, 25.9440618 48...",POINT (25.94422053876138 48.26803580531003),Chernivtsi Oblast
998,26743065.0,NaN,NaN,NaN,yes,109,NaN,NaN,Міська дитяча поліклініка,NaN,NaN,"POLYGON ((25.9424885 48.2675669, 25.942858 48....","POLYGON ((25.9424885 48.2675669, 25.942858 48....",POINT (25.94301388236103 48.26748416014559),Chernivtsi Oblast


In [23]:
buildings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6018387 entries, 0 to 6018386
Data columns (total 15 columns):
 #   Column             Dtype  
---  ------             -----  
 0   osm_id             float64
 1   buildingma         float64
 2   addrstreet         object 
 3   addrcity           object 
 4   building           object 
 5   addrhousen         object 
 6   office             object 
 7   addrfull           object 
 8   name               object 
 9   buildingle         object 
 10  source             object 
 11  geometry           object 
 12  geometry_wkt       object 
 13  geometry_wkt_cent  object 
 14  state              object 
dtypes: float64(2), object(13)
memory usage: 688.7+ MB


In [24]:
buildings_df.describe()

,osm_id,buildingma
count,6.018387e+06,0.0
mean,5.462761e+08,NaN
std,2.763411e+08,NaN
min,-1.388828e+07,NaN
25%,3.095529e+08,NaN
50%,5.065640e+08,NaN
75%,7.946179e+08,NaN
max,1.037134e+09,NaN


In [25]:
buildings_df_state = buildings_df[buildings_df["state"] == "Chernivtsi Oblast"]

In [26]:
buildings_df_state

,osm_id,buildingma,addrstreet,addrcity,building,addrhousen,office,addrfull,name,buildingle,source,geometry,geometry_wkt,geometry_wkt_cent,state
990,26742800.0,NaN,NaN,NaN,yes,91а,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9476411 48.2694433, 25.9477351 48...","POLYGON ((25.9476411 48.2694433, 25.9477351 48...",POINT (25.94778949260989 48.26940593569441),Chernivtsi Oblast
991,26742866.0,NaN,NaN,NaN,yes,103,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9453659 48.2687344, 25.9455505 48...","POLYGON ((25.9453659 48.2687344, 25.9455505 48...",POINT (25.94556896352371 48.26860415107055),Chernivtsi Oblast
992,26742872.0,NaN,NaN,NaN,yes,95,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9463234 48.2690174, 25.9465101 48...","POLYGON ((25.9463234 48.2690174, 25.9465101 48...",POINT (25.94652019729562 48.26888988885725),Chernivtsi Oblast
993,26742930.0,NaN,NaN,NaN,yes,97,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9462377 48.2685023, 25.9463827 48...","POLYGON ((25.9462377 48.2685023, 25.9463827 48...",POINT (25.94651171412588 48.26852193096251),Chernivtsi Oblast
994,26742953.0,NaN,NaN,NaN,yes,101,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9456757 48.2683202, 25.9458197 48...","POLYGON ((25.9456757 48.2683202, 25.9458197 48...",POINT (25.94594723050538 48.26834475095112),Chernivtsi Oblast
995,26742991.0,NaN,NaN,NaN,yes,107,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9442038 48.2683966, 25.9451136 48...","POLYGON ((25.9442038 48.2683966, 25.9451136 48...",POINT (25.94470209284609 48.26848820481244),Chernivtsi Oblast
996,26742992.0,NaN,Івана Миколайчука вулиця,NaN,yes,1,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9439751 48.2683212, 25.9442038 48...","POLYGON ((25.9439751 48.2683212, 25.9442038 48...",POINT (25.94413280550882 48.26830059335505),Chernivtsi Oblast
997,26742997.0,NaN,Івана Миколайчука вулиця,NaN,yes,3,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9440047 48.2681858, 25.9440618 48...","POLYGON ((25.9440047 48.2681858, 25.9440618 48...",POINT (25.94422053876138 48.26803580531003),Chernivtsi Oblast
998,26743065.0,NaN,NaN,NaN,yes,109,NaN,NaN,Міська дитяча поліклініка,NaN,NaN,"POLYGON ((25.9424885 48.2675669, 25.942858 48....","POLYGON ((25.9424885 48.2675669, 25.942858 48....",POINT (25.94301388236103 48.26748416014559),Chernivtsi Oblast
999,26743107.0,NaN,Івана Миколайчука вулиця,NaN,yes,2,NaN,NaN,NaN,2,NaN,"POLYGON ((25.9429389 48.2680376, 25.9430514 48...","POLYGON ((25.9429389 48.2680376, 25.9430514 48...",POINT (25.94312437031487 48.2680323818675),Chernivtsi Oblast


In [28]:
buildings_df_state["geometry_wkt_cent"].x

AttributeError: 'Series' object has no attribute 'x'

In [29]:
type(buildings_df_state["geometry_wkt_cent"])

pandas.core.series.Series

In [30]:
buildings_df_state

,osm_id,buildingma,addrstreet,addrcity,building,addrhousen,office,addrfull,name,buildingle,source,geometry,geometry_wkt,geometry_wkt_cent,state
990,26742800.0,NaN,NaN,NaN,yes,91а,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9476411 48.2694433, 25.9477351 48...","POLYGON ((25.9476411 48.2694433, 25.9477351 48...",POINT (25.94778949260989 48.26940593569441),Chernivtsi Oblast
991,26742866.0,NaN,NaN,NaN,yes,103,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9453659 48.2687344, 25.9455505 48...","POLYGON ((25.9453659 48.2687344, 25.9455505 48...",POINT (25.94556896352371 48.26860415107055),Chernivtsi Oblast
992,26742872.0,NaN,NaN,NaN,yes,95,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9463234 48.2690174, 25.9465101 48...","POLYGON ((25.9463234 48.2690174, 25.9465101 48...",POINT (25.94652019729562 48.26888988885725),Chernivtsi Oblast
993,26742930.0,NaN,NaN,NaN,yes,97,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9462377 48.2685023, 25.9463827 48...","POLYGON ((25.9462377 48.2685023, 25.9463827 48...",POINT (25.94651171412588 48.26852193096251),Chernivtsi Oblast
994,26742953.0,NaN,NaN,NaN,yes,101,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9456757 48.2683202, 25.9458197 48...","POLYGON ((25.9456757 48.2683202, 25.9458197 48...",POINT (25.94594723050538 48.26834475095112),Chernivtsi Oblast
995,26742991.0,NaN,NaN,NaN,yes,107,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9442038 48.2683966, 25.9451136 48...","POLYGON ((25.9442038 48.2683966, 25.9451136 48...",POINT (25.94470209284609 48.26848820481244),Chernivtsi Oblast
996,26742992.0,NaN,Івана Миколайчука вулиця,NaN,yes,1,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9439751 48.2683212, 25.9442038 48...","POLYGON ((25.9439751 48.2683212, 25.9442038 48...",POINT (25.94413280550882 48.26830059335505),Chernivtsi Oblast
997,26742997.0,NaN,Івана Миколайчука вулиця,NaN,yes,3,NaN,NaN,NaN,5,NaN,"POLYGON ((25.9440047 48.2681858, 25.9440618 48...","POLYGON ((25.9440047 48.2681858, 25.9440618 48...",POINT (25.94422053876138 48.26803580531003),Chernivtsi Oblast
998,26743065.0,NaN,NaN,NaN,yes,109,NaN,NaN,Міська дитяча поліклініка,NaN,NaN,"POLYGON ((25.9424885 48.2675669, 25.942858 48....","POLYGON ((25.9424885 48.2675669, 25.942858 48....",POINT (25.94301388236103 48.26748416014559),Chernivtsi Oblast
999,26743107.0,NaN,Івана Миколайчука вулиця,NaN,yes,2,NaN,NaN,NaN,2,NaN,"POLYGON ((25.9429389 48.2680376, 25.9430514 48...","POLYGON ((25.9429389 48.2680376, 25.9430514 48...",POINT (25.94312437031487 48.2680323818675),Chernivtsi Oblast


# calculate UTM

In [31]:
import geopandas as gpd
# change to GeoDataFrame
# geo_build_df = gpd.GeoDataFrame(buildings_df)
# geo_build_df = geo_build_df.set_geometry("geometry_wkt_cent")

In [32]:
# from pyproj import Proj
# pp = Proj(proj='utm',zone=10,ellps='WGS84', preserve_units=False)

# geo_build_df["xx"], geo_build_df["yy"]= pp(geo_build_df["geometry_wkt_cent"].x.values,geo_build_df["geometry_wkt_cent"].y.values)
# # My_data["X"] = xx
# # My_data["Y"] = yy 

In [33]:
# First, convert WKT column to geometries
#buildings_df_state['geometry_wkt_cent2'] = gpd.GeoSeries.from_wkt(buildings_df_state['geometry_wkt_cent'])

# Then create a GeoDataFrame, setting the geometry to the converted geometries
# geo_build_df = gpd.GeoDataFrame(buildings_df_state, geometry='geometry_wkt_cent')

In [80]:
# geo_build_df.crs = 'epsg:32635'
# # change the projection of geodf
# geo_build_df_2 = geo_build_df.to_crs(epsg=32635)
# print(geo_build_df_2)

         osm_id  buildingma                addrstreet addrcity building  \
990  26742800.0         NaN                       NaN      NaN      yes   
991  26742866.0         NaN                       NaN      NaN      yes   
992  26742872.0         NaN                       NaN      NaN      yes   
993  26742930.0         NaN                       NaN      NaN      yes   
994  26742953.0         NaN                       NaN      NaN      yes   
995  26742991.0         NaN                       NaN      NaN      yes   
996  26742992.0         NaN  Івана Миколайчука вулиця      NaN      yes   
997  26742997.0         NaN  Івана Миколайчука вулиця      NaN      yes   
998  26743065.0         NaN                       NaN      NaN      yes   
999  26743107.0         NaN  Івана Миколайчука вулиця      NaN      yes   

    addrhousen office addrfull                       name buildingle source  \
990        91а    NaN      NaN                        NaN          5    NaN   
991        103  

In [81]:
geo_build_df_2["geometry_wkt_cent"]

990    POINT (25.948 48.269)
991    POINT (25.946 48.269)
992    POINT (25.947 48.269)
993    POINT (25.947 48.269)
994    POINT (25.946 48.268)
995    POINT (25.945 48.268)
996    POINT (25.944 48.268)
997    POINT (25.944 48.268)
998    POINT (25.943 48.267)
999    POINT (25.943 48.268)
Name: geometry_wkt_cent, dtype: geometry

In [37]:
print(utm_crs)


EPSG:32635


In [37]:
print(utm_crs)


EPSG:32635


# Workspace 

In [5]:
war_fire_df_state = pd.DataFrame()

In [6]:
war_fire_df_state['geometry'] = war_fire_df.apply(lambda row: Point(row['LATITUDE'], row['LONGITUDE']), axis=1)

In [7]:
### with utm conversion


# First, find the UTM zone for the geometries and project to that CRS
# .estimate_utm_crs() is available in GeoPandas 0.9.0 and later
geo_warfires_df = gpd.GeoDataFrame(war_fire_df_state, geometry='geometry')
geo_warfires_df.set_crs(epsg=4326, inplace=True)
utm_crs = geo_warfires_df.estimate_utm_crs()
geo_warfires_df_utm = geo_warfires_df.to_crs("EPSG:32635")

# Then, apply the buffer in meters
buffer_radius_m = 500  # Buffer radius in meters
geo_warfires_df_utm['geometry_buff'] = geo_warfires_df_utm.buffer(buffer_radius_m)

geo_warfires_df_utm['geometry_buff']

0        POLYGON ((2795997.565 2854158.472, 2795995.157...
1        POLYGON ((2795602.983 2853982.924, 2795600.575...
2        POLYGON ((2796216.820 2853317.064, 2796214.412...
3        POLYGON ((2795253.774 2853919.160, 2795251.366...
4        POLYGON ((2796147.709 2853793.760, 2796145.302...
                               ...                        
42961    POLYGON ((2408284.856 4672156.064, 2408282.448...
42962    POLYGON ((2408366.943 4672068.513, 2408364.535...
42963    POLYGON ((2408640.113 4671426.694, 2408637.705...
42964    POLYGON ((2408666.684 4675501.440, 2408664.276...
42965    POLYGON ((2408573.729 4675364.182, 2408571.321...
Name: geometry_buff, Length: 42966, dtype: geometry